## Урок 14. Transfer learning/ ДЗ


**Задание**
1. Дообучить берт на задачу NER
2. Дообучить GPT на генерацию текста
3. Дообучить T5 на задачу суммаризации текста*



In [47]:
!pip install corus razdel

In [2]:
!pip install datasets evaluate transformers corus razdel seqeval transformers[torch] --quiet
!pip install accelerate -U --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00


In [3]:
!pip install -q datasets transformers seqeval corus razdel

In [4]:
!pip install evaluate

In [5]:
import numpy as np
import pandas as pd
import re

import evaluate
import logging
import torch

from datasets import Dataset, DatasetDict, load_dataset, load_metric
from evaluate import load as load_metric

from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForTokenClassification, AutoModelForCausalLM, T5ForConditionalGeneration
from transformers import DataCollatorForTokenClassification
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers.trainer import logger as noisy_logger

from corus import load_rudrec
from collections import Counter, defaultdict
from razdel import tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [6]:
model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 16

### Settings

In [24]:
device = 'cuda'

### Задание 1.

In [7]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2024-02-05 19:46:09--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2024-02-05 19:46:09--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json.1’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.02s   

2024-02-05 19:46:10 (79.2 MB/s) - ‘rudrec_annotated.json.1’ saved [1773014/1773014]



In [14]:
BERT_MODEL = 'cointegrated/rubert-tiny'
BERT_DATA = '/content/rudrec_annotated.json'
BERT_BATCH_SIZE = 16
BERT_EPOCHS = 10


**Data:**

In [15]:
def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [16]:
data = [extract_labels(item) for item in list(load_rudrec(BERT_DATA))]

data_train, data_test = train_test_split(data, test_size=0.1)
pd.DataFrame(data_train).head()

,tokens,tags
0,"[Ограничений, по, частоте, применения, нет, ,,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[Стоит, в, районе, 450, рублей, за, 6, таблето...","[O, O, O, O, O, O, O, B-Drugform, O]"
2,"[За, такую, сумму, можно, взять, ,, к, примеру...","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,"[Поэтому, после, 2, месяцев, лечение, все, поб...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR,..."
4,"[На, вкус, сладкие, приятные, ,, сладкие, ,, к...","[O, O, O, O, O, O, O, O, O, O]"


In [17]:
label_list = sorted({label for item in data_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list

label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [18]:
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)

def tokenize(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"],
                                 truncation=True,
                                 padding=True,
                                 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [19]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(data_train)),
    'test': Dataset.from_pandas(pd.DataFrame(data_test))
})
tokenized_data = ner_data.map(tokenize, batched=True)

Map:   0%|          | 0/4328 [00:00<?, ? examples/s]

Map:   0%|          | 0/481 [00:00<?, ? examples/s]

**Model:**

In [20]:
model = AutoModelForTokenClassification.from_pretrained(BERT_MODEL, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
for param in model.parameters():
    param.requires_grad = True

In [22]:
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [25]:
model = model.to(device)
tokenized_data = tokenized_data.with_format("torch", device=device)

args = TrainingArguments("ner",
                         evaluation_strategy = "epoch",
                         learning_rate=2e-5,
                         per_device_train_batch_size=BERT_BATCH_SIZE,
                         per_device_eval_batch_size=BERT_BATCH_SIZE,
                         num_train_epochs=BERT_EPOCHS,
                         weight_decay=0.01,
                         save_strategy='no',
                         report_to='none')

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(model,
                  args,
                  train_dataset=tokenized_data["train"],
                  eval_dataset=tokenized_data["test"],
                  data_collator=data_collator,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)

**Training:**

In [26]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.537523,0.597781,0.257621,0.360067,0.859466
2,0.706100,0.446094,0.553571,0.426181,0.481594,0.877385
3,0.706100,0.398679,0.622670,0.499103,0.554081,0.889829
4,0.406800,0.369977,0.609037,0.555888,0.581250,0.895802
5,0.406800,0.353284,0.611596,0.586372,0.598718,0.898872
6,0.340200,0.339640,0.635377,0.594740,0.614387,0.901361
7,0.340200,0.332224,0.613867,0.613867,0.613867,0.901692
8,0.314100,0.327479,0.621542,0.631201,0.626335,0.904181
9,0.314100,0.323024,0.657578,0.619845,0.638154,0.905923
10,0.287800,0.321884,0.646695,0.625822,0.636087,0.905343


TrainOutput(global_step=2710, training_loss=0.4015917141059228, metrics={'train_runtime': 131.0643, 'train_samples_per_second': 330.22, 'train_steps_per_second': 20.677, 'total_flos': 121258303108800.0, 'train_loss': 0.4015917141059228, 'epoch': 10.0})

In [27]:
trainer.evaluate()

{'eval_loss': 0.32188376784324646,
 'eval_precision': 0.6466954910438543,
 'eval_recall': 0.625821876867902,
 'eval_f1': 0.6360874848116648,
 'eval_accuracy': 0.90534262485482,
 'eval_runtime': 1.3468,
 'eval_samples_per_second': 357.145,
 'eval_steps_per_second': 23.018,
 'epoch': 10.0}

In [28]:
text = ' '.join(ner_data['test'][64]['tokens'])
tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
with torch.no_grad():
    pred = model(**tokens)

indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])

print(text)
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

Честно не понимаю , зачем таким маленьким детишкам выписывают капли на основе спирта .
[CLS]           O         
Ч               O         
##ест           O         
##но            O         
не              O         
по              O         
##нима          O         
##ю             O         
,               O         
за              O         
##чем           O         
таким           O         
мал             O         
##ень           O         
##ким           O         
дети            O         
##шка           O         
##м             O         
вы              O         
##пис           O         
##ывают         O         
ка              B-Drugform
##п             B-Drugform
##ли            B-Drugform
на              O         
основе          O         
с               O         
##пир           O         
##та            O         
.               O         
[SEP]           O         


### Задание 2.

In [31]:
GPT_MODEL = 'sberbank-ai/rugpt3small_based_on_gpt2'
GPT_DATA = '/content/all_recepies_inter.csv'
GPT_BATCH_SIZE = 16
GPT_EPOCHS = 3

**Data:**

In [32]:
data = pd.read_csv(GPT_DATA, sep='\t')
data.head()

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."


In [33]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [34]:
data = data.loc[:5000, 'Инструкции']
data_train, data_test = train_test_split(data, test_size=0.15)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

build_text_files(data_train, train_path)
build_text_files(data_test, test_path)

In [35]:
tokenizer = AutoTokenizer.from_pretrained(GPT_MODEL)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [36]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (741813 > 2048). Running this sequence through the model will result in indexing errors


**Model:**

In [37]:
model = AutoModelForCausalLM.from_pretrained(GPT_MODEL)
model.to(device)

args = TrainingArguments(output_dir="./gpt2-chief",
                         overwrite_output_dir=True,
                         num_train_epochs=GPT_EPOCHS,
                         per_device_train_batch_size=GPT_BATCH_SIZE,
                         per_device_eval_batch_size=GPT_BATCH_SIZE,
                         eval_steps = 400,
                         save_steps=800,
                         warmup_steps=500)

trainer = Trainer(model=model,
                  args=args,
                  data_collator=data_collator,
                  train_dataset=train_dataset,
                  eval_dataset=test_dataset)

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

**Training:**

In [38]:
trainer.train()

Step,Training Loss
500,2.450500
1000,2.216900


TrainOutput(global_step=1089, training_loss=2.31869236409281, metrics={'train_runtime': 622.1812, 'train_samples_per_second': 27.942, 'train_steps_per_second': 1.75, 'total_flos': 1135640494080000.0, 'train_loss': 2.31869236409281, 'epoch': 3.0})

In [39]:
prefix = "Берём сковородку "
tokens = tokenizer(prefix, return_tensors='pt').to(device)
size = tokens['input_ids'].shape[1]
res = model.generate(**tokens,
                     do_sample=False,
                     max_length=size+50,
                     repetition_penalty=5.,
                     temperature=0.5,
                     num_beams=10,)

decoded = tokenizer.decode(res[0])
result = decoded[len(prefix):]
print(prefix + result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Берём сковородку  и наливаем оливковое масло. Обжариваем его на среднем огне до золотистого цвета, затем добавляем мелко нарезанный чеснок (1-2 зубчика) и обжариваем ещё 2-3 минуты.  Добавляем в сковор


### Задание 3*.

In [40]:
from datasets import load_dataset

In [41]:
T5_MODEL = 'IlyaGusev/rut5_base_sum_gazeta'
T5_DATA = 'IlyaGusev/gazeta'
T5_BATCH_SIZE = 16
T5_EPOCHS = 5

**Data:**

In [42]:
data_train = load_dataset(T5_DATA, revision="v1.0", split='train[:10%]')
data_test = load_dataset(T5_DATA, revision="v1.0", split='test[:10%]')
data_train

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for IlyaGusev/gazeta contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/gazeta
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 5240
})

In [43]:
def len_tok(text):
    return len(text.split())

max_len_sum = max(map(len_tok,data_train['summary']))
max_len_tl = max(map(len_tok,data_train['title']))

tokenizer = AutoTokenizer.from_pretrained(T5_MODEL)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)
    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

data_train = data_train.map(tokenize, batched=True, batch_size=8)
data_test = data_test.map(tokenize, batched=True, batch_size=8)

data_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
data_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

tokenizer_config.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/5240 [00:00<?, ? examples/s]

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

**Model:**

In [44]:
output_dir = 'gazeta/output'
model = T5ForConditionalGeneration.from_pretrained(T5_MODEL).to(device)

args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=T5_EPOCHS,
    per_device_train_batch_size=T5_BATCH_SIZE,
    per_device_eval_batch_size=T5_BATCH_SIZE,
    eval_accumulation_steps=1,
    prediction_loss_only=True,
    learning_rate=0.00001,
    evaluation_strategy='steps',
    save_steps=1000,
    save_total_limit=1,
    remove_unused_columns=True,
    run_name='run_gazeta',
    logging_steps=500,
    eval_steps=500,
    logging_first_step=False,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=data_train,
    eval_dataset=data_test
)


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/977M [00:00<?, ?B/s]

**Training:**

In [45]:
trainer.train()

Step,Training Loss,Validation Loss
500,8.701100,2.973698
1000,4.499600,2.878594
1500,3.098900,2.769300


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=1640, training_loss=5.1715707174161585, metrics={'train_runtime': 742.8437, 'train_samples_per_second': 35.27, 'train_steps_per_second': 2.208, 'total_flos': 2608755379200000.0, 'train_loss': 5.1715707174161585, 'epoch': 5.0})

In [46]:
ID = 64
input_text = data_test['summary'][ID]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask,
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1,
        length_penalty=1,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print(input_text)
print(pred)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Жителей Миасса возмутило хвастовство мэра города Григория Тонких — чиновник опубликовал пост, в котором рассказал, как по его требованию в местной школе №23 дыры, которые ученики использовали в качестве туалетов, заменили на нормальные унитазы. На приложенном к новости снимке нового санузла видно, что между унитазами нет никакой перегородки — миассцы назвали это позором. При этом директор учебного заведения уверила, что кабины еще установят, а мэр «немного поторопился» с новостью.
Чиновники намерены заменить унитазы
